In [17]:
from sklearn.svm import SVR 
from yahoo_finance import Share 
import numpy as np 
import urllib
from bs4 import BeautifulSoup as bs


stock_code = input("What stock code do you want to predict? ")
past_days = input("Number of past days to input to model: ") 
future_days = input("Number of future days to predict: ")


def get_historical_data(name, number_of_days):
    data = []
    url = "https://finance.yahoo.com/quote/" + name + "/history/"
    rows = bs(urllib.request.urlopen(url).read(), "html5lib").findAll('table')[0].tbody.findAll('tr')

    for each_row in rows:
        divs = each_row.findAll('td')
        if divs[1].span.text  != 'Dividend':
            data.append({'Date': divs[0].span.text, 'Open': float(divs[1].span.text.replace(',',''))})

    return data[:number_of_days]


data = get_historical_data(stock_code, int(past_days))


stock_list = [] 


for i in data:
  stock_list.append((i['Open']))

What stock code do you want to predict? AAPL
Number of past days to input to model: 100
Number of future days to predict: 10


In [18]:
def predict_prices(prices, days_to_predict_in_future): 
    days = np.arange(0,len(stock_list), 1) 
    prices = (np.asarray(prices)).reshape(-1, 1)
    days = days.reshape(-1, 1)
    svr_rbf = SVR(kernel = 'rbf', C=1e3, gamma = 0.1) 
    svr_rbf.fit(days, prices.ravel()) 
    predict_days = (int(days_to_predict_in_future))
    max_day = 2 + max(days) + predict_days
    min_day = max(days) + 2 
    days_to_predict = np.arange(min_day, max_day, 1) 
    days_to_predict = days_to_predict.reshape(-1, 1)
    
    return svr_rbf.predict(days_to_predict)

print(predict_prices(stock_list, future_days))

[ 190.25198229  191.52517001  181.33650468  169.27420262  161.19864703
  157.43121032  156.12022297  155.76878183  155.69486097  155.6825199 ]


In [3]:
from textblob import TextBlob
from selenium import webdriver
import numpy as np

what_company = input("What company do you want sentiment analysis to be done on? ")
n_articles = input("On how many top articles? ")


search_term = str(what_company)
search = "http://uk.reuters.com/search/news?blob=" + search_term

driver = webdriver.Firefox()
driver.get(search)

number_of_articles = int(n_articles)
sentiment =[]

for i in range(number_of_articles):
    i = i + 1
    driver.get(search)
    link_url = "/html/body/div[5]/section[2]/div/div[1]/div[4]/div/div[3]/div[" + str(i) + "]/div/h3/a"
    elem = driver.find_element_by_xpath(link_url)
    elem.click()
    elements = driver.find_elements_by_xpath('/html/body/div[5]/div/div[1]/div[1]/div/div[2]/div[2]/div[1]/div')
    content = "".join([element.text for element in elements])
    blob = TextBlob(content)
    for sentence in blob.sentences:
        sentiment.append((sentence.sentiment.polarity))
    driver.back()

print("The sentiment surrounding " + search_term +  " is " + str(np.mean(sentiment)))


What company do you want sentiment analysis to be done on? apple
On how many top articles? 10


WebDriverException: Message: 'geckodriver' executable needs to be in PATH. 
